## Wumpus KB

In [1]:
import gym
import fh_ac_ai_gym

wumpus_env = gym.make('Wumpus-v0')

KB Agent with CNF

In [9]:




class KB:
    def __init__(self):
        self.visited = {}
        self.rules = []
        self.rules.append('-P00')
        self.rules.append('-W00')
        self.wumpus_env = wumpus_env
    def run(self):
        t = 0
        step = ""
        self.wumpus_env.reset()
        obs, reward,done, info = self.wumpus_env.step(1)
        while(step != "exit"):
            
            self.wumpus_env.render()
            self.identify_event(obs)
            
            ask = input("ASK: ")
            if ask!="n":
                self.ask(ask)
            
            step = input("What do you want to do? ")
            if(step=="walk" or step=='w'):                    
                obs, reward,done, info = self.wumpus_env.step(0)
                
            elif (step=='turn left' or step=='l'):
                obs, reward,done, info = self.wumpus_env.step(1)
            elif (step=='turn right' or step=='r'):
                obs, reward,done, info = self.wumpus_env.step(2)
            elif (step=='grab' or step=='g'):
                obs, reward,done, info = self.wumpus_env.step(3)
            elif (step=='shoot' or step=='s'):
                obs, reward,done, info = self.wumpus_env.step(4)
            elif (step=='climb' or step=='c'):
                obs, reward,done, info = self.wumpus_env.step(5)
            elif (step!='exit' and step!='q' and step!='e' and step.isdigit()):
                if(int(step)<6 and int(step)>0):
                    obs, reward,done, info = self.wumpus_env.step(int(step))
            else:
                break
                
                
            print(obs)
            print(info)
            t = t+1
        print(self.rules)
            
    def identify_event(self, obs):
        pos = '' + str(obs['x']) + str(obs['y'])
        print(pos)
        print(self.visited)
        if pos in self.visited.keys():
            return
        self.tell('-P' + str(obs['x']) + str(obs['y']) + '')
        self.tell('-W' + str(obs['x']) + str(obs['y']) + '')
        if(obs['breeze'] == True):
            self.tell('B' + str(obs['x']) + str(obs['y']) + '')
            self.add_toKB('B', 'P', str(obs['x']), str(obs['y']))
        else:
            self.add_toKB('B', 'P', str(obs['x']), str(obs['y']))
            self.tell('-B' + str(obs['x']) + str(obs['y']) + '')
        
        if(obs['stench'] == True):
            self.tell('S' + str(obs['x']) + str(obs['y']) + '')
            self.add_toKB('S', 'W', str(obs['x']), str(obs['y']))
        else:
            self.tell('-S' + str(obs['x']) + str(obs['y']) + '')
            self.add_toKB('S', 'W', str(obs['x']), str(obs['y']))
        self.visited.update({str(pos) : 'visited'})
                
    
    def tell(self, rule):
        self.rules.append(rule)
        
    def add_toKB(self, event, wp, posx, posy):
        
        surr = self.get_surrounding_fields(int(posx), int(posy))
        p = event + str(posx) + str(posy)
        not_p = '-' + p
        
        new_rule = not_p
        for key in surr:
            surr_posx = key
            surr_posy = surr[key]
            new_rule = new_rule + ',' + wp + str(surr_posx) + str(surr_posy)
            # -B00,P10,P01
    
        self.tell(new_rule)
        
        for key in surr:
            surr_posx = key
            surr_posy = surr[key]
            rule = '-' + wp + str(surr_posx) + str(surr_posy) + ',' + p
            self.tell(rule)
            # -P10,B00
            # -P01,B00
    
    def ask(self, alpha):
        safe = self.PL_Resolution(alpha)
        print(safe)
    
    # Benachbarte Felder
    def get_surrounding_fields(self, posx, posy):
        surr = {}
        if posx>=1:
            surr.update({posx-1 : posy})
        if posy>=1:
            surr.update({posx : posy-1})
            
        surr.update({posx+1 : posy})
        surr.update({posx : posy+1})
        
        return surr
        
    def PL_Resolution(self, alpha):
        clauses = set(self.rules)
        if alpha[0] == '-':
               clauses.add((alpha[1:])) 
        else:
            clauses.add(('-' + alpha))
        
        print(clauses)
        
        while len(clauses)>0:
            new = []
            for i in clauses:
                for j in clauses:
                    resolvents = self.PLResolve(i, j)
                    if resolvents == "empty":
                        return True
                    
                    if resolvents != "":
                        new.append(resolvents)
                    
                    flag = 0
            if(all(x in clauses for x in new)): 
                flag = 1
            if flag == 1:
                return False
                    
            clauses.update(new)
            #print(clauses)
            
        return False
        
        
    def PLResolve(self, Ci, Cj):
        ci_splitted = set(Ci.split(','))
        cj_splitted = set(Cj.split(','))
        
        
        for i in ci_splitted:
            for j in cj_splitted:
                if(i[0] == '-'):
                    if i[1:] == j:
                        ci_splitted.remove(i)
                        cj_splitted.remove(j)
                        
                        resolvent_set = ci_splitted.union(cj_splitted)
                        
                        list_of_strings = [str(s) for s in resolvent_set]
                        resolvent = ",".join(list_of_strings)
                        #print(resolvent)
                        if len(resolvent)==0:
                            return "empty"
                        return resolvent
                        
                elif i == j[1:] and j[0] == '-':
                    ci_splitted.remove(i)
                    cj_splitted.remove(j)
                    
                    resolvent_set = ci_splitted.union(cj_splitted)
                    
                    list_of_strings = [str(s) for s in resolvent_set]
                    resolvent = ",".join(list_of_strings)
                    #print(resolvent)
                    if len(resolvent)==0:
                        return "empty"
                    return resolvent
        #print('nothing found')
        return ""  
        

        
KB = KB()
KB.run()

+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |  G|   |
|   |   |   |   |
+---+---+---+---+
|   |   |W  |   |
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
| A^|   |   |   |
+---+---+---+---+
Perception [ St: False, Br: True, G: False, Bu: False, Sc: False ]
Score : -1
00
{}
ASK: n
What do you want to do? w
{'x': 0, 'y': 1, 'gold': False, 'direction': <Direction.NORTH: 0>, 'arrow': True, 'stench': False, 'breeze': False, 'glitter': False, 'bump': False, 'scream': False}
{'no further information', 'info'}
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |  G|   |
|   |   |   |   |
+---+---+---+---+
|   |   |W  |   |
| A^|   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
Perception [ St: False, Br: False, G: False, Bu: False, Sc: False ]
Score : -2
01
{'00': 'visited'}
ASK: P10
{'-P10,B00', '-S01', '-P00', '-W10,S00', '-P01,B00', '-S00,W10,W01', '-W11,S01', '-S01,W02,W11', '-W00', 

ask perfomance gets bad for many actions on the board

Example from the lecture, with the given set of rules the KB can proof if there is no Pit at (0,1)

In [7]:
KB.rules = []
KB.rules.append('-P10,B00')
KB.rules.append('-B00,P01,P10')
KB.rules.append('-P01,B00')
KB.rules.append('-B00')
print(KB.rules)
KB.ask('-P01')

['-P10,B00', '-B00,P01,P10', '-P01,B00', '-B00']
{'P01', '-P10,B00', '-B00,P01,P10', '-P01,B00', '-B00'}
True
